# Guidelines to reading this notebook

- This notebook is divided into following key sections 
    - PREPROCESS : Pre-processing & clean data on data to extract relevant features. 
    
    - MODEL : Train a model what can give recommendations. We have created 2 models. One to train over cat_item & other
     to train over RBC Line Item title (LIT). cat_item is much cleaner than RBC LIT & forms a large chunk of tickets we          recommend (approximately 75 %) . For tickets that don't have cat_item, we use the RBC Line Item Title as the key feature. 
     
    - POST PROCESS : Filter the results to ensure analyst are assigned tickets based on the teams they work for. The analyst      have less then or equal to 45 tickets in their queue. The analyst is not on a vacation. 
    
- For every function I specify what the function does. This is followed by input parameters (params) takes by the functions & 
  output it returns. 

In [7]:
# Below are the python modules we would be using. 
import numpy as np
import pandas as pd
import os, glob, sys , re
import sklearn , pickle
from datetime import datetime
from dateutil import relativedelta
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

# Environment Variables

Below are the various locations where we look for various files, which have the data required by the bot. You need make sure you set these paths appropriately. 

In [8]:
os.environ['PROJECT_DIR'] = "C:\\Users\\581686284\\PycharmProjects\\Automated_Ticket_Management-2\\"
os.environ['SM9_TRAINING_DATA_EXCEL'] = "rawdata\\sm9\\train_excel\\*.xlsx"
os.environ['SM9_TEST_DATA_EXCEL'] = "rawdata\\sm9\\test_excel\\*.xlsx"
os.environ['SERVICE_NOW_TRAINING_DATA_EXCEL'] = "rawdata\\serviceNow\\train_excel\\*.xlsx"
os.environ['SERVICE_NOW_TEST_DATA_EXCEL'] = "rawdata\\serviceNow\\test_excel\\*.xlsx"
os.environ['RECOMMENDATIONS'] = "recommendations\\"

# Utility Functions

Utility Methods that are required. Below is a short description of what it does & why is it needed.



In [9]:
# object_to_int : Used when model is pre-processed. Converts float (.0) & NaN's to integer for Analyst Employee Id. Analyst Id's 
# are numbers , not floating point. If number is not an integer, we're presently setting to an arbitrary value. Such values would 
# be removed later as we only consider analyst that are part of the team & have been assigned tickets. 

def object_to_int(x):
    try:
        return int(x)
    except ValueError:
        return 123

In [11]:
# getAnalyst : Gives all the analyst we have in ESAM Operations as per the 'Team List.xlsx' file. We remove FTE Team Lead, as they 
# don't work on tickets. This method returns employee id's as an index , along with analyst's name, skills  & status. We'll
# refer to this as ESAM Operations Roaster, to avoid mixing it with the various internal teams (e.g : Windows, Insurance etc)
# we have. 

def getAnalyst():
    team_data_columns = ['Name','Status','Employee #','Windows & Appliciation (Line of Business) Skillset']
    team_sheet = pd.read_excel(os.environ['PROJECT_DIR'] + 'rawdata\\Team List.xlsx' , header=1 , usecols = team_data_columns, index_col = 2)
    team_sheet_without_leads = team_sheet.loc[team_sheet['Status'].str.strip() != 'FTE Team Lead']
    analyst_emp_id = team_sheet_without_leads.index.tolist()
    return team_sheet_without_leads

In [12]:
# Setting team_sheet_without_leads as a global variable as this information is required by multiple functions. 
team_sheet_without_leads = getAnalyst()
# team_dict = getTeams()
# team_dict

# Pre-process Data

## SM9

In [100]:
# Need to pass it a parameter to recognize whether we're dealing with training or test data. 
# This function reads the SM9 data we have. Focuses on columns listed in sm9_columns. Its MANDATORY to ensure the columns are 
# listed exactly this way in the excels we read. We're using excel over csv, as it takes less space & we have a size constraint,
# when deploying on PCF Cloud. We pre-process the data to focus only on tickets in Windows & Applications bin. Only those tickets
# that have been assigned to analyst, so that the bot can learn from it. We ensure the analyst employee id's are integers, not 
# floats. We filter the tickets to focus only on tickets that have been assigned to analyst part of the current roaster & not 
# others who are part of the team. Hence its critical we have all analyst working with the team in the ESAM Roaster.  
# We assign tickets based on teams an analyst is part of, we use Client Base to map to the appropriate team. If its tickets raised
# for Windows bin & Client Base is empty, then we set the Client Base as Windows. This makes it possible to assigns tickets 
# raised for Windows bin to analyst working in Windows Team. For some tickets the RBC Line Item Title is missing. In such as case
# we set the RBC Line Item Title to the RBC Description. We use RBC Line Item Title to make recommendations for tickets that 
# dont have a cat_item value (cat_item is present in Service Now). 

# Input Params : This function takes as input the location of the directory where it should look for fetch SM9 data (both Windows & Applications)
# Output : The SM9 tickets which have RITM & those which don't have RITM. 

def getSM9TrainData(location):
    #sm9_files = glob.glob(os.environ['PROJECT_DIR'] +  location)
    print('Started getSM9TrainData')
    sm9_files = glob.glob(location)
    sm9_columns = ['Assigned Dept', 'Assigned to', 'Number','RBC Line Item Title', 'RBCMMPRITM', 'Rbc Description', 'Client Base']
    # Read tickets in Excel file
    sm9_data_list = [ pd.read_excel(sm9_file) for sm9_file in sm9_files ]
    sm9_data = pd.concat(sm9_data_list)
    sm9_data = sm9_data[sm9_columns]
    sm9_data = sm9_data.reset_index(drop=True)
    # Consider only Windows & Application tickets
    sm9_data = sm9_data.loc[sm9_data['Assigned Dept'].isin(['EAA_WINDOWS SERVICES_IMPL' , 'EAA_APPLICATION SERVICES_IMPL'])]
    # Remove those tickets that don't have Analyst linked to it. Noise. 
    sm9_data = sm9_data[pd.notnull(sm9_data['Assigned to'])]
    # Convert objects (float values in particular) to int
    sm9_data['Assigned to'] = sm9_data['Assigned to'].apply(object_to_int)
    # Give me only those tickets, who are assigned to analyst who are part of the current team. 
    analyst_emp_id = team_sheet_without_leads.index.tolist()
    sm9_data = sm9_data.loc[sm9_data['Assigned to'].isin(analyst_emp_id)]
    # If RBC Line Item Title is null , then set replace it with RBC Description. 
    tickets_with_no_title = sm9_data.loc[sm9_data['RBC Line Item Title'].isnull()].index.tolist()
    sm9_data.loc[tickets_with_no_title,'RBC Line Item Title'] = sm9_data.loc[tickets_with_no_title,'Rbc Description']
    sm9_data_without_RITM = sm9_data.loc[sm9_data['RBCMMPRITM'].isnull()]
    sm9_data_with_RITM = sm9_data.loc[~sm9_data['RBCMMPRITM'].isnull()]
    print('Done getSM9TrainData')
    return sm9_data_with_RITM , sm9_data_without_RITM


In [101]:
def getSM9TestData(location):
    print('Started getSM9TestData')
    sm9_files = glob.glob(location)
    sm9_columns = ['Assigned Dept', 'Number','RBC Line Item Title', 'RBCMMPRITM', 'Rbc Description', 'Client Base']
    #sm9_data_list = [ pd.read_csv(sm9_file , encoding='latin-1' , usecols=sm9_columns ) for sm9_file in sm9_files ]
    # Read tickets in CSV file
    #sm9_data_list = [ pd.read_csv(sm9_file , encoding='latin-1' ) for sm9_file in sm9_files ]
    # Read tickets in Excel file
    sm9_data_list = [ pd.read_excel(sm9_file) for sm9_file in sm9_files ]
    sm9_data = pd.concat(sm9_data_list)
    sm9_data = sm9_data[sm9_columns]
    sm9_data = sm9_data.reset_index(drop=True)
    # Consider only Windows & Application tickets
    sm9_data = sm9_data.loc[sm9_data['Assigned Dept'].isin(['EAA_WINDOWS SERVICES_IMPL' , 'EAA_APPLICATION SERVICES_IMPL'])]
    # Set client base as Windows for tickets that have null Client Base & Assigned Dept as EAA_WINDOWS SERVICES_IMPL
    win_tickets_null_client_base = sm9_data.loc[(sm9_data['Client Base'].isnull()) & (sm9_data['Assigned Dept'] == 'EAA_WINDOWS SERVICES_IMPL')].index.tolist()
    sm9_data.loc[win_tickets_null_client_base , 'Client Base'] = 'Windows'
    # If RBC Line Item Title is null , then set replace it with RBC Description. 
    tickets_with_no_title = sm9_data.loc[sm9_data['RBC Line Item Title'].isnull()].index.tolist()
    sm9_data.loc[tickets_with_no_title,'RBC Line Item Title'] = sm9_data.loc[tickets_with_no_title,'Rbc Description']
    sm9_data_without_RITM = sm9_data.loc[sm9_data['RBCMMPRITM'].isnull()]
    sm9_data_with_RITM = sm9_data.loc[~sm9_data['RBCMMPRITM'].isnull()]
    print('Ended getSM9TestData')
    return sm9_data_with_RITM , sm9_data_without_RITM

In [102]:
# This method clean 
def cleanTicketTitle(data):
    print('Started cleanTicketTitle')
    data['RBC Line Item Title'] = data['RBC Line Item Title'].str.replace(r"\d+/\d+",'')
    data['RBC Line Item Title'] = data['RBC Line Item Title'].str.replace(r"[0-9]",'')
    #sm9_data_without_RITM['RBC Line Item Title'] = sm9_data_without_RITM['RBC Line Item Title'].str.replace(r"[0-9]",'')
    data['RBC Line Item Title'] = data['RBC Line Item Title'].str.replace(r"A termination request has recently been submitted on behalf of RBC Access Manager for .*",'termination request Access Manager')
    data['RBC Line Item Title'] = data['RBC Line Item Title'].str.replace(r"Access Manager manual termination request submitted .*",'Access Manager manual termination')
    data['RBC Line Item Title'] = data['RBC Line Item Title'].str.replace(r"An access review has recently been completed .*",'access review completed')
    data['RBC Line Item Title'] = data['RBC Line Item Title'].str.replace(r'[:.,-/?*[\]&+()!]',' ')
    data['RBC Line Item Title'] = data['RBC Line Item Title'].str.replace(r'( ESC | PA | PO | PC | PD| Apr | April )','')
    data['RBC Line Item Title'] = data['RBC Line Item Title'].str.replace(r'  ',' ')
    data['RBC Line Item Title'] = data['RBC Line Item Title'].str.strip()
    print('Ended cleanTicketTitle')
    return data



## Service Now

In [103]:
# Service Now does not have Client Base. How are we going to proceed once Service Now goes live. 
# Changes to be made once Service Now is up : 

In [104]:
# Need to pass it a parameter to recognize whether we're dealing with training or test data. 
def getServiceNowTrainData(location):
    print('Started getServiceNowTrainData')
    #sn_files = glob.glob(os.environ['PROJECT_DIR'] +  location)
    sn_files = glob.glob(location)
    #sn_files
    sn_columns = ['cat_item' , 'number' , 'u_assignment_group', 'state' , 'stage' ]
#     reqd_stages = ['Fulfillment']
#     reqd_states = ['Work in Progress','Open']
    #sn_df = [pd.read_csv(sn_file , encoding='latin-1' , usecols=sn_columns) for sn_file in sn_files]
    sn_df = [pd.read_excel(sn_file , usecols=sn_columns) for sn_file in sn_files]
    sn_data = pd.concat(sn_df)
    sn_data = sn_data.loc[sn_data['u_assignment_group'].isin(['EAA_WINDOWS SERVICES_IMPL','EAA_APPLICATION SERVICES_IMPL'])]
    #sn_data = sn_data.loc[sn_data['state'].str.strip().isin(reqd_states) | sn_data['stage'].str.strip().isin(reqd_stages)]
    print('Ended getServiceNowTrainData')
    return sn_data
    #sn_data.head()

In [105]:
# Filter Service Now data by 

In [106]:
def getServiceNowTestData(location):
    print('Started getServiceNowTestData')
    sn_files = glob.glob(location)
    #sn_files
    sn_columns = ['cat_item' , 'number' , 'u_assignment_group', 'state' , 'stage' ]
    reqd_stages = ['Fulfillment']
    reqd_states = ['Work in Progress','Open']
    #sn_df = [pd.read_csv(sn_file , encoding='latin-1' , usecols=sn_columns) for sn_file in sn_files]
    sn_df = [pd.read_excel(sn_file , usecols=sn_columns) for sn_file in sn_files]
    sn_data = pd.concat(sn_df)
    sn_data = sn_data.loc[sn_data['u_assignment_group'].isin(['EAA_WINDOWS SERVICES_IMPL','EAA_APPLICATION SERVICES_IMPL'])]
    sn_data = sn_data.loc[sn_data['state'].str.strip().isin(reqd_states) | sn_data['stage'].str.strip().isin(reqd_stages)]
    print("sn_data.shape : ", sn_data.shape)
    print('Ended getServiceNowTestData')
    return sn_data
    #sn_data.head()

### Merge Data

In [107]:
def mergeTrainingData(sm9_data_with_RITM,sn_data):
#     sm9_data_with_RITM , sm9_data_without_RITM = getSM9TrainData()
#     sn_data = getServiceNowTrainData()
    print('Started mergeTrainingData')
    merged_data = pd.merge(sn_data ,sm9_data_with_RITM, left_on = 'number' , right_on='RBCMMPRITM' , how='outer', indicator=True)
    # Only consider tickets assigned to analyst. Remove tickets that are auto-completed. 
    merged_data = merged_data.loc[~merged_data['Assigned to'].isnull()]
    # Convert Assigned to from float to int. 
    merged_data['Assigned to'] = merged_data['Assigned to'].apply(object_to_int)
    # You might need RBC Description for cases when RBC Line Item Title is null. 
    merged_data = merged_data[['number' , 'Number' , 'cat_item' , 'u_assignment_group' , 'Assigned to' , 'RBC Line Item Title', 'Rbc Description' , 'Client Base' , 'state' , 'stage']]
    merged_data_without_cat_item = merged_data.loc[merged_data['cat_item'].isnull()]
    merged_data_with_cat_item = merged_data.loc[~merged_data['cat_item'].isnull()]
    
    
    
    #merged_data_by_title = mergeDataByRBCTitle(sm9_data_without_RITM,merged_data_without_cat_item)
    print('Ended mergeTrainingData')
    return merged_data_with_cat_item , merged_data_without_cat_item

In [108]:
def mergeTestData(sm9_data_with_RITM,sn_data):
#     sm9_data_with_RITM , sm9_data_without_RITM = getSM9TrainData()
#     sn_data = getServiceNowTrainData()
    print('Started mergeTestData')
    merged_data = pd.merge(sn_data ,sm9_data_with_RITM, left_on = 'number' , right_on='RBCMMPRITM' , how='outer', indicator=True)
    # Only consider tickets assigned to analyst. Remove tickets that are auto-completed. 
    #merged_data = merged_data.loc[~merged_data['Assigned to'].isnull()]
    # Convert Assigned to from float to int. 
    #merged_data['Assigned to'] = merged_data['Assigned to'].apply(object_to_int)
    # You might need RBC Description for cases when RBC Line Item Title is null. 
    merged_data = merged_data[['Assigned Dept','number' , 'Number' , 'cat_item' , 'u_assignment_group' , 'RBC Line Item Title' , 'Rbc Description' , 'Client Base', 'state' , 'stage' ]]
    merged_data_without_cat_item = merged_data.loc[merged_data['cat_item'].isnull()]
    merged_data_with_cat_item = merged_data.loc[~merged_data['cat_item'].isnull()]
    #merged_data_by_title = mergeDataByRBCTitle(sm9_data_without_RITM,merged_data_without_cat_item)
    
    # Set client base as Windows for tickets that have null Client Base & Assigned Dept as EAA_WINDOWS SERVICES_IMPL
    win_tickets_null_client_base = merged_data_with_cat_item.loc[(merged_data_with_cat_item['Client Base'].str.strip().isnull()) & ((merged_data_with_cat_item['Assigned Dept'].str.strip() == 'EAA_WINDOWS SERVICES_IMPL') | (merged_data_with_cat_item['u_assignment_group'].str.strip() == 'EAA_WINDOWS SERVICES_IMPL'))].index.tolist()
    merged_data_with_cat_item.loc[win_tickets_null_client_base , 'Client Base'] = 'Windows'
    
    # Set client base as Windows for tickets that have null Client Base & Assigned Dept as EAA_WINDOWS SERVICES_IMPL
    win_tickets_null_client_base = merged_data_without_cat_item.loc[(merged_data_without_cat_item['Client Base'].str.strip().isnull()) & ((merged_data_without_cat_item['Assigned Dept'].str.strip() == 'EAA_WINDOWS SERVICES_IMPL') | (merged_data_with_cat_item['u_assignment_group'].str.strip() == 'EAA_WINDOWS SERVICES_IMPL'))].index.tolist()
    merged_data_without_cat_item.loc[win_tickets_null_client_base , 'Client Base'] = 'Windows'
    
    print('merged_data_with_cat_item : ', merged_data_with_cat_item.shape)
    print('merged_data_without_cat_item : ', merged_data_without_cat_item.shape)
    print('Ended mergeTestData')
    return merged_data_with_cat_item , merged_data_without_cat_item

In [109]:
def prepareDataByRBCTitle(sm9_data_without_RITM ,merged_data_without_cat_item):
    print('Started prepareDataByRBCTitle')
    sm9_data_without_RITM = cleanTicketTitle(sm9_data_without_RITM)
    merged_data_without_cat_item = cleanTicketTitle(merged_data_without_cat_item)
    print('Ended prepareDataByRBCTitle')
    return sm9_data_without_RITM , merged_data_without_cat_item

# Train

In [110]:
# features : X ; labels : Y ; filename : model name ; location : where model is stored. 
# Try using joblib to persist the model. 
def trainModel(features , labels, model , countVec):
    print('Started trainModel')
    cv = CountVectorizer(ngram_range=(2, 2) , stop_words='english')
    X = cv.fit_transform(features)
    Y = labels
    #len(cv.get_feature_names())
    # Removed stratify=Y as the least populated class has only 1 sample. 
    # X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.03, random_state=42)
    #print(X_train.shape , Y_train.shape , X_test.shape , Y_test.shape)
    clf = MultinomialNB()
    clf = MultinomialNB().fit(X, Y)
    # save the model to disk
    pickle.dump(clf, open(model, 'wb'))
    # save count vectorizer to disk
    pickle.dump(cv, open(countVec, 'wb'))
    # predict_proba = clf.predict_proba(X_test)
    print('Ended trainModel')
    return clf

## Predict

In [111]:
# TO-DO : Use joblib to load the model
def predictModelByCatItem(features , model , countVec):
    # Code to preprocess the data
    # Code to load the model 
    print('Started predictModelByCatItem')
    print("features : ", features.shape)
    cv = pickle.load(open(countVec, 'rb'))
    clf = pickle.load(open(model, 'rb'))
    
    #cv = CountVectorizer(ngram_range=(2, 2) , stop_words='english')
    #cv = CountVectorizer(ngram_range=(2, 2) )
    X = cv.transform(features)
    
    predict_proba = clf.predict_proba(X)
    analyst_prob = []
    for p_p in predict_proba:
        analyst_prob.append(zip(clf.classes_ , p_p))
    sorted_analyst_prob = []
    for lpp in analyst_prob : 
        sorted_analyst_prob.append(sorted(lpp, key=lambda x: x[1], reverse=True ))
    print('Ended predictModelByCatItem')
    return sorted_analyst_prob
    # Code for model evaluation. Not relevant at this point 
#     index_match = []
#     for i,slp in enumerate(sorted_analyst_prob) :
#         index_match.append([lb[0] for lb in slp].index(Y_test.iloc[i]))
#     index_match

In [112]:
# TO-DO : Use joblib to load the model
def predictModelByTitle(features , model , countVec):
    # Code to preprocess the data
    # Code to load the model 
    print('Started predictModelByTitle')
    print("features : ", features.shape)
    cv = pickle.load(open(countVec, 'rb'))
    clf = pickle.load(open(model, 'rb'))
    
    
    #cv = CountVectorizer(ngram_range=(2, 2) , stop_words='english')
    X = cv.transform(features)
    #clf = pickle.load(open(filename_location, 'rb'))
    predict_proba = clf.predict_proba(X)
    analyst_prob = []
    for p_p in predict_proba:
        analyst_prob.append(zip(clf.classes_ , p_p))
    sorted_analyst_prob = []
    for lpp in analyst_prob : 
        sorted_analyst_prob.append(sorted(lpp, key=lambda x: x[1], reverse=True ))
    print('Started predictModelByTitle')
    return sorted_analyst_prob
    # Code for model evaluation. Not relevant at this point 
#     index_match = []
#     for i,slp in enumerate(sorted_analyst_prob) :
#         index_match.append([lb[0] for lb in slp].index(Y_test.iloc[i]))
#     index_match

# Post-process Recommendations

### Teams

These are teams we have in ESAM that work on tickets 

In [113]:
def getTeams():
    print('Started getTeams')
    from datetime import datetime
    analyst_teams = pd.read_excel(os.environ['PROJECT_DIR'] +  '\\rawdata\\Global 2018 Operations Schedule.xlsx', sheetname='MONTHLY SCHEDULE', header = 2 , index_col=0)
    team_name = []
    team_index = analyst_teams.index.tolist()
    for t in team_index:
        if type(t) is str:
            team_name.append(t)
    # Get the month for which we want the team for . 
    current_month = datetime.now().strftime('%B') # 'August'
    team_dict = {} # Dictionary of team name as key & team mates as value. 
    for t in range(len(team_name) - 1):
        team = analyst_teams.loc[team_name[t]:team_name[t+1] , current_month]
        team = team.iloc[:len(team) - 1]
        team.reset_index(drop=True,inplace=True)
        team = team.loc[~team.isnull()]
        team = team[:len(team)]
        team_dict[team_name[t]] = team
    print('Ended getTeams')
    return team_dict

In [114]:
getTeams()

Started getTeams
Ended getTeams


{'CDN I&TS': 0    MacLellan, Jon 
 1     Stone, Randall
 Name: August, dtype: object,
 'Cards & Payments, RBC Bank': 0    Mostoles, Sherylle
 1    Jette, Christopher
 Name: August, dtype: object,
 'Caribbean': 0            King, Sancha
 1    Dubissette, Donnette
 2          Dookie, Pamela
 3      Adderley, Nikeisha
 4        Fonrose, Sadicki
 Name: August, dtype: object,
 'Dominion Securities, E&T, PH&N': 0     Papagiannidis, Kostas
 1            Chohan, Sophia
 2         Lilbourne, Lauren
 3    MariaFrancis, Jennifia
 Name: August, dtype: object,
 'Early': 0    McQuarrie, Katrina
 Name: August, dtype: object,
 'Insurance': 0      Gibbons, Paul
 1    Jereza, Agustin
 Name: August, dtype: object,
 'Late': 0    Kadoura, Zaki
 1      Shah, Rohan
 Name: August, dtype: object,
 'Mailbox/ Passwords': Series([], Name: August, dtype: object),
 'UK I&TS': Series([], Name: August, dtype: object),
 'UK Wealth Mgmt': 0     Johnston, Dan
 1    Hardisty, Erin
 2       Wrona, Anna
 3     Dorney, Niam

### Client Base to Teams Mapping

In [115]:
def getClientBaseToTeamMapping():
    print('Started getClientBaseToTeamMapping')
    client_base_to_team = pd.read_excel(os.environ['PROJECT_DIR'] +  '\\rawdata\\Client_Base_to_Team_Mapper.xlsx',index_col=0)
    print('Ended getClientBaseToTeamMapping')
    return client_base_to_team


### Vacation Schedule

In [116]:
# Get Vacation Schedule: Returns the vacation calender for current month. 
# Used to filter out analyst on leave. 

def getVacationSchedule():
    print('Started getVacationSchedule')
    all_vacation = pd.read_excel(os.environ['PROJECT_DIR'] +  '\\rawdata\\Vacation Calendar 2018.xlsx', sheetname='All', index_col=0)
    all_vacation.columns = list(range(1,32))
    # Get the month & year. This would be used to get vacation information for the corresponding month and year. 
    current_month = datetime.now().strftime('%B') # 'August'
    current_year = datetime.now().strftime('%Y')  # 2018
    month_year = (current_month + ' ' + str(current_year)).upper()
    next_month_year = ''
    vc = pd.DataFrame()
    # Make sure the year in the below condition is a variable. What if the year is December 2019. Then the below code would fail. 
    if month_year != 'DECEMBER 2018':
        #nextmonth = datetime.date.today() + relativedelta.relativedelta(months=1)
        nextmonth = datetime.now() + relativedelta.relativedelta(months=1)
        next_month = nextmonth.strftime('%B')
        next_month_year = (next_month + ' ' + current_year).upper()
        vc = all_vacation.loc[month_year : next_month_year]
    else:
        # What if the month year is December 2018
        # In this case you would use the current month_year variable.
        vc = all_vacation.loc[month_year : ]
        pass
    #print("To : " , next_month_year)    
    print('Ended getVacationSchedule')
    return vc



In [117]:
vc = getVacationSchedule()

Started getVacationSchedule
Ended getVacationSchedule


In [118]:
# Code to check if an analyst is on vacation 

# Based on current day & the analyst name, check if the analyst is on a vacation. 

def isAnalystOnLeave(analyst_name):
#     print('Started isAnalystOnLeave')
    from datetime import datetime
    leave_days = [7.5 , 7.50 , 'L' , 'C' , 'T']
    current_day = int(datetime.now().strftime('%d'))
    
    #analyst_name = 'Papagiannidis, Kostas'
#     print("analyst_name : ", analyst_name)
#     print("current_day : ", current_day)
#     print("type(current_day) : " , type(current_day))
#     print("current_day.astype(int) : " , current_day.astype(int))
    vc
    try:
        if vc.loc[analyst_name][current_day] in leave_days : 
            #print("Yes")
#             print('Ended isAnalystOnLeave : True')
            print(analyst_name + ' is on leave. No tickets would be assigned.')
            return True
        else:
#             print('Ended isAnalystOnLeave : False')
            return False
    except Exception as e:
        #print("Exceptin in isAnalystOnLeave : " , e)
        return False
        


### Availability

In [119]:
# Get all tickets in an analyst's queue. 
# Some files use csv , where as others use .xlsx . Lets standardize to excel sheets as they occupy less space. Check & ensure 
# the same applies on the linux box too
def getAvailabilityPerAnalyst():
    print('Started getAvailabilityPerAnalyst')
    availability_columns = ['Assigned to','Pending Customer','Client Base']
    #availability = pd.read_csv(os.environ['PROJECT_DIR'] +  '\\rawdata\\availability.csv' , encoding='latin-1' , usecols=availability_columns )
    availability = pd.read_excel(os.environ['PROJECT_DIR'] +  '\\rawdata\\availability.xlsx' , usecols=availability_columns )
    availability = availability.loc[~availability['Assigned to'].isnull()]
    availability['Pending Customer'] = availability['Pending Customer'].fillna('FALSE')
    availability['Assigned to'] = availability['Assigned to'].apply(object_to_int)
    #availability.loc[availability['Assigned to'].isin(['310294822'])]
    availability_grouped = availability.groupby(['Assigned to','Pending Customer']).count()
    availability_grouped.rename(columns = {'Client Base' : 'No_of_Tickets_Assigned'} , inplace=True)
    #availability_grouped
    # Below line is commented as we made team_sheet_without_leads global
    #team_sheet_without_leads = getAnalyst()
    analyst_emp_id = team_sheet_without_leads.index.tolist()
    tickets_analyst_queue = {}
    for analyst in analyst_emp_id:
        try:
            tickets_analyst_queue[analyst] = availability_grouped.loc[(analyst,'FALSE')]['No_of_Tickets_Assigned']
        except Exception as e:
            print("Exception in getAvailabilityPerAnalyst : " , e)
            tickets_analyst_queue[analyst] = 0
    print('Availability')
    print(tickets_analyst_queue)
    print('Ended getAvailabilityPerAnalyst')
    return tickets_analyst_queue

In [120]:
def postProcess(sorted_analyst_prob, ticket):
    # This variable will save a list of recommendations for each ticket
    print('Started postProcess')
#     print("ticket.head() : ")
#     print(ticket)
    #ticket.head()
    r1 , r2 , r3 , r4 = [] , [] , [] ,[]
    recommendations_list = []
    client_base_to_team = getClientBaseToTeamMapping()
    team_dict = getTeams()
#     print("team_dict")
#     print(team_dict)
#     print("team_sheet_without_leads")
#     print(team_sheet_without_leads)
    #tickets_to_recommend = ticket.index.tolist()
    tickets_analyst_queue = getAvailabilityPerAnalyst()
    count = 0
#     for count , t in enumerate(ticket):
    for index, t in ticket.iterrows():
#         print("t  " , t)
#         print("t['Client Base'] :  ", t['Client Base'])
        c_b = t['Client Base']
#         print("c_b : ", c_b)
#         print("type(c_b) : ", type(c_b))
        # TO-DO What if the client base is null
        #if c_b != np.nan:
        if pd.isnull(c_b):
            t_l = ['All']
        else:
            team = client_base_to_team.loc[c_b,'ESAM Application Teams ']
            t_l = team.split('|')
        # Get me all the analyst to be considered for this ticket
        a_l = []
        if t_l[0] in 'All':
            a_l = team_sheet_without_leads['Name'].tolist()
            # Consider all analyst across all teams
            # iterate through all keys of team_list dictionary. Set a_l to the list of analyst across teams. 
        else:
            for team in t_l:
                team = team.strip()
                for t_m in team_dict[team]:
                    a_l.append(t_m)
            # We now have all analyst we need to consider. 
#             print(a_l)
        # Check Vacation Calender & remove those on Vacation
        # Use List comprehension insead of the below line
        analyst_not_on_vacation = []
        for a in a_l:
            if isAnalystOnLeave(a):
                continue
            else:
                analyst_not_on_vacation.append(a)
        #analyst_not_on_vacation
        # Getting employee id based on analyst name. 
        analyst_not_on_leave = []
        # Below line is commented as we made team_sheet_without_leads global
        #team_sheet_without_leads = getAnalyst()
        for a in analyst_not_on_vacation:
            try:
                emp_id =(team_sheet_without_leads.loc[team_sheet_without_leads['Name'].str.strip() == a.strip()]).index.tolist()[0]
                analyst_not_on_leave.append(emp_id)
            except Exception as e:
                print("Exception in postprocess while looping over analyst_not_on_vacation" , e)
                print("a :",a)
#                 print("team_dict : ", team_dict)
#                 print("team_sheet_without_leads : ",team_sheet_without_leads)
            #print(emp_id)
            
        #analyst_not_on_leave
        # Get probability of analyst to be considered , sort them in descending order. It already sorted. Evaluate your approach
        # Do you need to iterate through it OR you can pick the analyst. Some thought needed. 
        analyst_prob = []
        for i in range(len(sorted_analyst_prob[count])):
            if sorted_analyst_prob[count][i][0] in analyst_not_on_leave:
                analyst_prob.append(sorted_analyst_prob[count][i])
        #analyst_prob  
        # Check availability & don't consider analyst having more than 45 tickets. Need to amend this later to give lower probability to 
        # those having more tickets in their bin. Once you have 4 analyst who can resolve such tickets & are available.
        analyst_avail = {}
        for a in analyst_prob:
            try:
                analyst_avail[a[0]] = tickets_analyst_queue[a[0]]
            except Exception as e:
                print("Exception in postprocess while looping over analyst_prob ", e)
                analyst_avail[a[0]] = -1
            #tickets_analyst_queue
        #analyst_avail
        recommendations = []
        for a_p in analyst_prob:
            if analyst_avail[a_p[0]] <= 45:
                recommendations.append(team_sheet_without_leads.loc[a_p[0]]['Name'])
        recommendations_list.append(recommendations)
        count = count + 1

    # Add recommendations to the tickets 
    print('Size of Recommendations_list : ',len(recommendations_list))
    print('Number of tickets in shape : ', ticket.shape)
    for r in recommendations_list:
        try:
            r1.append(r[0])
        except Exception as e:
            r1.append('None 1')
            print("r1.append(r[0]) : ", e)
        try:
            r2.append(r[1])
        except Exception as e:
            r2.append('None 2')
            print("r2.append(r[1]) : ", e)
        try:
            r3.append(r[2])
        except Exception as e:
            r3.append('None 3')
            print("r3.append(r[2]) : ", e)
        try:
            r4.append(r[3])
        except Exception as e:
            r4.append('None 4')
            print("r3.append(r[3]) : ", e)   
        
            
    
    R1 = pd.Series(r1 , index=ticket.index)    
    R2 = pd.Series(r2 , index=ticket.index)    
    R3 = pd.Series(r3 , index=ticket.index)    
    R4 = pd.Series(r4 , index=ticket.index)
#     print('R4 : ', R4)

    ticket['R1'] = R1
    ticket['R2'] = R2
    ticket['R3'] = R3
    ticket['R4'] = R4 
    print('Ended postProcess')
    return ticket

# Predict

In [121]:
# Useful code that would comes in use when we have the team data frame. Used to get name to the employee based on id & vice-versa

# emp_id = (team_sheet_without_leads.loc[team_sheet_without_leads['Name'] == 'Aberin, Mario']).index.tolist()
# name = team_sheet_without_leads.loc[191918473]['Name'] 

In [122]:
def train():
    # Pre-process Data 
    print('Started train')
    sm9_data_with_RITM , sm9_data_without_RITM = getSM9TrainData(os.environ['PROJECT_DIR'] + os.environ['SM9_TRAINING_DATA_EXCEL'])
    sn_data = getServiceNowTrainData(os.environ['PROJECT_DIR'] + os.environ['SERVICE_NOW_TRAINING_DATA_EXCEL'])
    merged_data_with_cat_item , merged_data_without_cat_item = mergeTrainingData(sm9_data_with_RITM,sn_data)
    sm9_data_without_RITM , merged_data_without_cat_item = prepareDataByRBCTitle(sm9_data_without_RITM,merged_data_without_cat_item)
#     sm9_data_without_RITM = sm9_data_without_RITM[['RBC Line Item Title','Assigned to']]
#     merged_data_without_cat_item = merged_data_without_cat_item[['RBC Line Item Title','Assigned to']]
    tickets_by_title = pd.concat([sm9_data_without_RITM , merged_data_without_cat_item])
    clf_by_cat_item = trainModel(merged_data_with_cat_item['cat_item'],merged_data_with_cat_item['Assigned to'] , os.environ['PROJECT_DIR'] + "models\\by_cat_item.pkl",os.environ['PROJECT_DIR'] + "models\\count_vec_cat_item.pkl")
    clf_by_title = trainModel(tickets_by_title['RBC Line Item Title'],tickets_by_title['Assigned to'],os.environ['PROJECT_DIR'] + "models\\by_title.pkl", os.environ['PROJECT_DIR'] + "models\\count_vec_by_title.pkl")
    print('Ended train')
#     return tickets_by_title
    # train models & save them 
    

In [123]:
import datetime
def predict():
    # Pre-process Data 
    print('Started predict')
    sm9_data_with_RITM , sm9_data_without_RITM = getSM9TestData(os.environ['PROJECT_DIR'] + os.environ['SM9_TEST_DATA_EXCEL'])
    sn_data = getServiceNowTestData(os.environ['PROJECT_DIR'] + os.environ['SERVICE_NOW_TEST_DATA_EXCEL'])
    merged_data_with_cat_item , merged_data_without_cat_item = mergeTestData(sm9_data_with_RITM,sn_data)
    sm9_data_without_RITM , merged_data_without_cat_item = prepareDataByRBCTitle(sm9_data_without_RITM,merged_data_without_cat_item)
#     sm9_data_without_RITM = sm9_data_without_RITM[['RBC Line Item Title']]
#     merged_data_without_cat_item = merged_data_without_cat_item[['RBC Line Item Title']]
    tickets_by_title = pd.concat([sm9_data_without_RITM , merged_data_without_cat_item])
    sorted_analyst_prob_by_item = predictModelByCatItem(merged_data_with_cat_item['cat_item'],os.environ['PROJECT_DIR'] + "models\\by_cat_item.pkl",os.environ['PROJECT_DIR'] + "models\\count_vec_cat_item.pkl")
    cat_item_ticket_recommendations = postProcess(sorted_analyst_prob_by_item,merged_data_with_cat_item)
    sorted_analyst_prob_by_title = predictModelByTitle(tickets_by_title['RBC Line Item Title'],os.environ['PROJECT_DIR'] + "models\\by_title.pkl",os.environ['PROJECT_DIR'] + "models\\count_vec_by_title.pkl")
    rbc_title_ticket_recommendations = postProcess(sorted_analyst_prob_by_title,tickets_by_title)
    ticket_recommendations = pd.concat([cat_item_ticket_recommendations,rbc_title_ticket_recommendations])
    filename = os.environ['PROJECT_DIR'] + os.environ['RECOMMENDATIONS'] + "tickets_recommendations_" + datetime.datetime.today().strftime('%Y-%m-%d') + ".xlsx"
#     filename = "tickets_recommendation_" + datetime.datetime.now().isoformat() + ".xlsx"
    # Prefarably save as excel 
    #ticket_recommendations.to_csv(path_or_buf=filename  , encoding="Latin-1" , index = False)
    #ticket_recommendations.to_excel(path_or_buf=filename , index = False)
    # Save as excel 
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    ticket_recommendations.to_excel(writer,sheet_name='Sheet1',index = False)
    writer.save()
    print('Ended predict')
    # Done saving recommendations as excel. 
    # Send email to email list. 
    
#     os.environ['SERVICE_NOW_TRAINING_DATA'] = "rawdata\\serviceNow\\train\\*.csv"
#     os.environ['SERVICE_NOW_TEST_DATA'] = "rawdata\\serviceNow\\test\\*.csv"
    # Load Models & pass data
    # Post-process recommendations

In [1]:
#train()
#predict()

In [2]:
# my_list = [['Aazim', 'Shiraz'],['Mom','Dad'],['Bhabhi','Divine']]
# names = []
# for l in range(len(my_list)):
#     print(my_list[l])

In [3]:
# import datetime
# #now=datetime.datetime.now()
# #now.isoformat()
# print('fsfsds ' + datetime.datetime.now().isoformat() + ' ffewfwe')

In [4]:
# import pandas as pd
# import datetime

# # # Create a Pandas dataframe from some data.
# df = pd.DataFrame({'Data': [10, 20, 30, 20, 15, 30, 45]})

# # # Create a Pandas Excel writer using XlsxWriter as the engine.
# # writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')
# filename = os.environ['PROJECT_DIR'] + os.environ['RECOMMENDATIONS'] + "tickets_recommendations_" + datetime.datetime.today().strftime('%Y-%m-%d') + ".xlsx"
# print(filename)
# writer = pd.ExcelWriter(filename , engine='xlsxwriter')
# df.to_excel(writer,sheet_name='Sheet1' )
# writer.save()

# # Convert the dataframe to an XlsxWriter Excel object.
# df.to_excel(writer, sheet_name='Sheet1')

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()

In [5]:
# import datetime
# d = datetime.datetime.now()
# datetime.datetime.today().strftime('%Y-%m-%d')

In [6]:
# import os

# from flask import Flask, request, render_template, send_from_directory

# #__author__ = 'ibininja'

# app = Flask(__name__)

# APP_ROOT = os.path.dirname(os.path.abspath(__file__))

# print(APP_ROOT)

In [7]:
import os

#os.path.abspath()
os.getcwd()

'C:\\Users\\581686284\\PycharmProjects\\Automated_Ticket_Management-2\\jupyter'

In [8]:
# os.path.abspath('') 

In [9]:
# APP_ROOT = os.path.dirname(os.path.abspath(''))
# APP_ROOT
# filename = 'otpa_fs_32313'
# filename_to_target = {
#                             'sm9_learn' : 'data/sm9/train',
#                             'sm9_predict' : 'data/sm9/test',
#                             'sn_learn' : 'data/serviceNow/train',
#                             'sn_predict' : 'data/serviceNow/test',
#                             'otpa' : 'data/postprocess/otpa',
#                             'vacation calendar' : 'data/postprocess',
#                             'global' : 'data/postprocess',
#                             'team list' : 'data/postprocess',
#                             'client_base_to_team_mapper' : 'data/postprocess'
#                          }
# file_type = [key for key in filename_to_target.keys() if filename.lower().startswith(key)]
# # print(file_type.pop() )
# # print(file_type)
# print('filename_to_target : ',  filename_to_target[file_type.pop()])

In [10]:
!pip freeze

alabaster==0.7.10
anaconda-client==1.6.5
anaconda-navigator==1.6.8
anaconda-project==0.8.0
aniso8601==3.0.2
asn1crypto==0.22.0
astroid==1.5.3
astropy==2.0.2
babel==2.5.0
backports.shutil-get-terminal-size==1.0.0
beautifulsoup4==4.6.0
bitarray==0.8.1
bkcharts==0.2
blaze==0.11.3
bleach==2.0.0
bokeh==0.12.7
boto==2.48.0
boto3==1.7.29
botocore==1.10.29
Bottleneck==1.2.1
bz2file==0.98
CacheControl==0.12.3
certifi==2017.7.27.1
cffi==1.10.0
chardet==3.0.4
click==6.7
cloudpickle==0.4.0
clyent==1.2.2
colorama==0.3.9
comtypes==1.1.2
conda==4.3.27
conda-build==3.0.22
conda-verify==2.0.0
contextlib2==0.5.5
cryptography==2.0.3
cycler==0.10.0
Cython==0.26.1
cytoolz==0.8.2
dask==0.15.2
datashape==0.5.4
decorator==4.1.2
distlib==0.2.5
distributed==1.18.3
docutils==0.14
entrypoints==0.2.3
et-xmlfile==1.0.1
fastcache==1.0.2
filelock==2.0.12
Flask==0.12.2
Flask-Cors==3.0.3
Flask-Jsonpify==1.5.0
flask-marshmallow==0.9.0
Flask-RESTful==0.3.6
gensim==3.4.0
gevent==1.2.2
glob2==0.5
greenlet==0.4.12
h5py==2.7

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [11]:
import pip
for package in pip.get_installed_distributions():
    name = package.project_name # SQLAlchemy, Django, Flask-OAuthlib
    print(name)
    key = package.key # sqlalchemy, django, flask-oauthlib
    module_name = package._get_metadata("top_level.txt") # sqlalchemy, django, flask_oauthlib
    location = package.location # virtualenv lib directory etc.
    version = package.version # ve

zict
xlwt
xlwings
XlsxWriter
xlrd
wrapt
wincertstore
win-unicode-console
win-inet-pton
widgetsnbextension
wheel
Werkzeug
webencodings
wcwidth
urllib3
unicodecsv
typing
traitlets
tornado
toolz
testpath
tblib
tables
sympy
statsmodels
SQLAlchemy
spyder
sphinxcontrib-websupport
Sphinx
sortedcontainers
sortedcollections
snowballstemmer
smart-open
six
singledispatch
simplegeneric
setuptools
seaborn
scipy
scikit-learn
scikit-image
s3transfer
ruamel-yaml
rope
retrying
requests
QtPy
qtconsole
QtAwesome
pyzmq
PyYAML
pywin32
PyWavelets
pytz
python-dateutil
pytest
PySocks
pyparsing
pyOpenSSL
pyodbc
pylint
Pygments
pyflakes
pycurl
pycrypto
pycparser
pycosat
pycodestyle
py
psutil
prompt-toolkit
progress
ply
plotly
pkginfo
pip
Pillow
pickleshare
pep8
patsy
pathlib2
path.py
partd
pandocfilters
pandas
packaging
openpyxl
olefile
odo
numpydoc
numpy
numexpr
numba
notebook
nose
nltk
networkx
nbformat
nbconvert
navigator-updater
multipledispatch
msgpack-python
mpmath
mistune
menuinst
mccabe
matplotlib
marsh

In [12]:
import os
dirname = os.path.dirname(__file__)
filename = os.path.join(dirname, 'relative/path/to/file/you/want')

NameError: name '__file__' is not defined